new code


In [1]:
import pandas as pd
import numpy as np

# Load dataset (update the path if needed)
data = pd.read_csv(r"C:\Users\Eeksha\Desktop\GSECD-final-project1.0\version2\civic-data-forge-main\supabase\data\dataset1.csv")

# --- Column names (copy these exactly from your file) ---
gdp_col = "GDP growth (annual %)"
infl_col = "Inflation, consumer prices (annual %)"
unemp_col = "Unemployment, total (% of total labor force) (modeled ILO estimate)"
credit_col = "Domestic credit to private sector (% of GDP)"
exp_col = "Exports of goods and services (% of GDP)"
imp_col = "Imports of goods and services (% of GDP)"

# --- Ensure numeric (coerce non-numeric to NaN) ---
for col in [gdp_col, infl_col, unemp_col, credit_col, exp_col, imp_col]:
    data[col] = pd.to_numeric(data[col], errors="coerce")

# --- Handle missing values ---
# Option A (recommended): keep NaN and treat missing as not triggering the flag.
# Option B: fill with reasonable values, e.g., forward-fill or median. Uncomment if desired.
# data.fillna(method="ffill", inplace=True)    # example forward-fill
# data.fillna(data.median(numeric_only=True), inplace=True)

# --- Compute crisis score (count of triggered conditions) ---
data["Crisis_Score"] = (
    (data[gdp_col] < 0).astype(int) +                             # contraction
    (data[infl_col] > 20).astype(int) +                           # serious inflation
    (data[unemp_col] > 10).astype(int) +                          # high unemployment
    (data[credit_col] > 100).astype(int) +                        # excessive private credit
    (data[exp_col] < 15).astype(int) +                            # weak exports
    ((data[imp_col] - data[exp_col]) > 20).astype(int)            # large trade gap
)

# --- Create binary Crisis_Flag: 1 if >=2 triggers, else 0 ---
data["Crisis_Flag"] = (data["Crisis_Score"] >= 2).astype(int)

# Optional: categorize severity
def categorize(score):
    if score >= 4:
        return "Severe"
    elif score >= 2:
        return "Moderate"
    elif score == 1:
        return "Low"
    else:
        return "None"

data["Crisis_Level"] = data["Crisis_Score"].apply(categorize)

# Preview
cols_to_show = ["Country Name", "Date", gdp_col, infl_col, unemp_col, "Crisis_Score", "Crisis_Flag", "Crisis_Level"]
print(data[cols_to_show].head(20))

# Save to csv
out_fn = "economic_dataset_crisis_label.csv"
data.to_csv(out_fn, index=False)
print(f"\n📂 File saved as: {out_fn}")

   Country Name  Date  GDP growth (annual %)  \
0       Albania  2000               7.462859   
1       Albania  2001               8.863731   
2       Albania  2002               4.628396   
3       Albania  2003               5.333264   
4       Albania  2004               5.266262   
5       Albania  2005               5.130822   
6       Albania  2006               6.018981   
7       Albania  2007               6.500093   
8       Albania  2008               6.907062   
9       Albania  2009               2.690752   
10      Albania  2010               2.973154   
11      Albania  2011               2.463517   
12      Albania  2012               0.984130   
13      Albania  2013               1.707228   
14      Albania  2014               2.240227   
15      Albania  2015               2.227704   
16      Albania  2016               3.908936   
17      Albania  2017               3.283176   
18      Albania  2018               3.671419   
19      Albania  2019               2.06

In [2]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, classification_report
# import shap
# import warnings

# # -------------------------
# # Config
# # -------------------------
# INPUT_CSV = "/content/economic_dataset_crisis_label.csv"
# OUT_PRED_CSV = "future_crisis_predictions.csv"
# RANDOM_STATE = 42
# TEST_SIZE = 0.2
# PROB_THRESHOLD = 0.90
# N_ESTIMATORS = 200

# # -------------------------
# # 1) Load dataset
# # -------------------------
# data = pd.read_csv(INPUT_CSV)
# data["Date"] = data["Date"].astype(int)

# # Create numeric feature list (exclude target/metadata)
# exclude_cols = ["Country Name", "Country Code", "Date", "Crisis_Flag", "Crisis_Level", "Crisis_Score"]
# numeric_features = data.drop(columns=exclude_cols, errors='ignore').select_dtypes(include=[np.number]).columns.tolist()

# # -------------------------
# # 2) Lag features (use last-year indicators)
# # -------------------------
# data = data.sort_values(["Country Name", "Date"]).reset_index(drop=True)

# for col in numeric_features:
#     data[f"{col}_lag1"] = data.groupby("Country Name")[col].shift(1)

# # Target: next-year crisis
# data["Crisis_NextYear"] = data.groupby("Country Name")["Crisis_Flag"].shift(-1)

# # Drop rows with missing lag or target
# lagged_cols = [f"{c}_lag1" for c in numeric_features]
# data_clean = data.dropna(subset=lagged_cols + ["Crisis_NextYear"]).copy()
# data_clean["Crisis_NextYear"] = data_clean["Crisis_NextYear"].astype(int)

# # -------------------------
# # 3) Prepare X, y
# # -------------------------
# X = data_clean[lagged_cols].fillna(data_clean[lagged_cols].median())
# y = data_clean["Crisis_NextYear"]
# meta = data_clean[["Country Name", "Date"]] # Remove Country Code

# # -------------------------
# # 4) Train/test split
# # -------------------------
# if len(np.unique(y)) == 1:
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
# else:
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

# # -------------------------
# # 5) Train RandomForest
# # -------------------------
# model = RandomForestClassifier(n_estimators=N_ESTIMATORS, random_state=RANDOM_STATE, class_weight="balanced")
# model.fit(X_train, y_train)

# # -------------------------
# # 6) Evaluate model
# # -------------------------
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"\n✅ Model Accuracy: {accuracy*100:.2f}%")
# print(classification_report(y_test, y_pred, zero_division=0))

# # -------------------------
# # 7) SHAP explainer
# # -------------------------
# explainer = shap.TreeExplainer(model)
# sv_future = explainer.shap_values(X)[1]  # class=1

# # -------------------------
# # 8) Predict for next year
# # -------------------------
# last_year = data["Date"].max()
# future_rows = data[data["Date"] == last_year].copy()
# if future_rows.empty:
#     raise ValueError(f"No rows for Date == {last_year}")

# # Build X_future from lagged features
# X_future = future_rows[lagged_cols].fillna(X.median())
# future_probs = model.predict_proba(X_future)[:, 1]

# # SHAP values for future
# sv_future_rows = explainer.shap_values(X_future)
# if isinstance(sv_future_rows, list) and len(sv_future_rows) > 1:
#     shap_future_rows = np.array(sv_future_rows[1])
# else:
#     shap_future_rows = np.array(sv_future_rows)


# # -------------------------
# # 9) Prepare CSV with reason indicators
# # -------------------------
# results = []
# for idx in range(len(future_rows)):
#     country = future_rows.iloc[idx]["Country Name"]
#     prob = future_probs[idx]

#     # Only save countries above threshold
#     if prob >= PROB_THRESHOLD:
#         raw_shap = shap_future_rows[idx]
#         # Flatten nested list if needed
#         if isinstance(raw_shap[0], (list, np.ndarray)):
#             instance_shap = np.array([float(x[0]) for x in raw_shap])
#         else:
#             instance_shap = np.array(raw_shap, dtype=float)

#         # Top 3 features
#         feat_vals = X_future.iloc[idx].values
#         feat_names = X_future.columns.tolist()
#         feat_pairs = list(zip(feat_names, instance_shap, feat_vals))
#         top_feats = sorted(feat_pairs, key=lambda x: abs(x[1]), reverse=True)[:3]
#         reason = ", ".join([f"{f[0].replace('_lag1','')}={f[2]:.2f}" for f in top_feats])

#         results.append({
#             "Country Name": country,
#             "Predicted_Year": last_year+1,
#             "Crisis_Prob": prob,
#             "Reason_Indicators": reason
#         })

# # Save to CSV
# if results:
#     pd.DataFrame(results).to_csv(OUT_PRED_CSV, index=False)
#     print(f"\n📂 High-risk countries saved with reason indicators: {OUT_PRED_CSV}")
# else:
#     print("\n🌍 No countries exceeded the crisis probability threshold.")

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import shap

# ===============================
# 1. Load dataset
# ===============================
data = pd.read_csv(r"C:\Users\Eeksha\Desktop\GSECD-final-project1.0\version2\civic-data-forge-main\supabase\models\economic_dataset_crisis_label.csv")
data["Date"] = data["Date"].astype(int)


# Features and target
X = data.drop(columns=["Crisis_Flag", "Country Name", "Date", "Country Code", "Crisis_Level", "Crisis_Score"], errors="ignore")
y = data["Crisis_Flag"]
country_year = data[["Country Name", "Date"]]

# Split dataset
X_train, X_test, y_train, y_test, cy_train, cy_test = train_test_split(
    X, y, country_year, test_size=0.2, random_state=42, stratify=y
)

# ===============================
# 2. Train Ensemble Model
# ===============================
model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced")
model.fit(X_train, y_train)

# Evaluate accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n✅ Model Accuracy: {accuracy * 100:.2f}%\n")

# ===============================
# 3. Predict for Next Year (2025)
# ===============================
latest_year = data["Date"].max()
future_data = data[data["Date"] == latest_year].copy()
future_data["Date"] = latest_year + 1  # simulate 2025

X_future = future_data.drop(columns=["Crisis_Flag", "Country Name", "Date", "Country Code", "Crisis_Level", "Crisis_Score"], errors="ignore")
future_probs = model.predict_proba(X_future)[:, 1]

# Add predictions
future_data["Crisis_Probability"] = future_probs

# ===============================
# 4. Identify High-Risk Countries
# ===============================
threshold = 0.80  # show countries with ≥80% crisis probability
high_risk = future_data[future_data["Crisis_Probability"] >= threshold]

if high_risk.empty:
    print("🌍 No countries exceed the crisis probability threshold.")
else:
    # SHAP explanation
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_future) # This will return a list of arrays

    # Select the SHAP values for the positive class (class 1)
    if isinstance(shap_values, list) and len(shap_values) > 1:
        shap_values_positive = shap_values[1]
    else:
        shap_values_positive = shap_values

    # Flatten nested list if needed
    # Check if the shape is (n_instances, n_features, n_classes)
    if len(shap_values_positive.shape) == 3:
        # Select the SHAP values for the positive class (index 1) for each feature
        shap_values_positive_flat = shap_values_positive[:, :, 1]
    else:
        # Assume it's already in the shape (n_instances, n_features)
        shap_values_positive_flat = shap_values_positive


    # Convert shap_values to a DataFrame to align with X_future and then high_risk
    shap_df = pd.DataFrame(shap_values_positive_flat, columns=X_future.columns, index=X_future.index)
    high_risk_shap = shap_df.loc[high_risk.index]


    print("\n🔥 High-Risk Countries (≥80% Crisis Probability)\n")
    for idx in high_risk.index:
        country = high_risk.loc[idx, "Country Name"]
        prob = high_risk.loc[idx, "Crisis_Probability"] * 100

        # Get feature importance for this country
        instance_shap = high_risk_shap.loc[idx].values
        feature_contribs = sorted(
            zip(X_future.columns, instance_shap, X_future.loc[idx]),
            key=lambda x: abs(x[1]),
            reverse=True
        )[:3]

        print(f"🚨 {country} — Crisis Probability: {prob:.2f}%")
        for feat, contrib, val in feature_contribs:
            print(f"   {feat:<45} Contribution: {contrib:+.3f} | Value: {val:.2f}")
        print("-" * 70)


✅ Model Accuracy: 97.24%


🔥 High-Risk Countries (≥80% Crisis Probability)

🚨 Botswana — Crisis Probability: 97.00%
   Unemployment, total (% of total labor force) (modeled ILO estimate) Contribution: +0.311 | Value: 23.14
   GDP growth (annual %)                         Contribution: +0.276 | Value: -2.99
   Domestic credit to private sector (% of GDP)  Contribution: -0.041 | Value: 32.93
----------------------------------------------------------------------
🚨 Burundi — Crisis Probability: 82.50%
   Exports of goods and services (% of GDP)      Contribution: +0.341 | Value: 10.80
   Unemployment, total (% of total labor force) (modeled ILO estimate) Contribution: -0.136 | Value: 0.90
   Inflation, consumer prices (annual %)         Contribution: +0.123 | Value: 20.21
----------------------------------------------------------------------
🚨 Gambia, The — Crisis Probability: 91.00%
   Exports of goods and services (% of GDP)      Contribution: +0.516 | Value: 6.55
   Unemployment, total

final_dataset_GSECD

In [7]:
import joblib
joblib.dump(model, 'economic_crisis_model.pkl') 

['economic_crisis_model.pkl']

In [3]:
import pickle

# Save model and feature order
model_data = {
    "model": model,
    "feature_order": list(X.columns)
}

with open("economic_model.pkl", "wb") as f:
    pickle.dump(model_data, f)

print("✅ Saved economic_model.pkl with feature order:", model_data["feature_order"])


✅ Saved economic_model.pkl with feature order: ['Domestic credit to private sector (% of GDP)', 'Exports of goods and services (% of GDP)', 'GDP growth (annual %)', 'GDP per capita (current US$)', 'Gross fixed capital formation (% of GDP)', 'Imports of goods and services (% of GDP)', 'Inflation, consumer prices (annual %)', 'Unemployment, total (% of total labor force) (modeled ILO estimate)']


In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score

print("\n" + "="*30)
print("📊 FINAL MODEL EVALUATION METRICS")
print("="*30)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

# Try to get ROC-AUC if probabilities are available
try:
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_prob)
    else:
        roc_auc = "N/A (No probability scores)"
except:
    roc_auc = "N/A"

# Print results
print(f"✅ Accuracy:  {accuracy:.4f}")
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall:    {recall:.4f}")
print(f"✅ F1 Score:  {f1:.4f}")
print(f"✅ ROC-AUC:   {roc_auc}")
print("\n🔍 Confusion Matrix:")
print(conf_matrix)
print("\n📄 Classification Report:")
print(classification_report(y_test, y_pred, zero_division=0))
print("="*30)


📊 FINAL MODEL EVALUATION METRICS
✅ Accuracy:  0.9724
✅ Precision: 0.9653
✅ Recall:    0.8797
✅ F1 Score:  0.9205
✅ ROC-AUC:   0.995275387569336

🔍 Confusion Matrix:
[[707   5]
 [ 19 139]]

📄 Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       712
           1       0.97      0.88      0.92       158

    accuracy                           0.97       870
   macro avg       0.97      0.94      0.95       870
weighted avg       0.97      0.97      0.97       870

